In [23]:
%load_ext aiida
%aiida

The aiida extension is already loaded. To reload it, use:
  %reload_ext aiida


Loaded AiiDA DB environment - profile name: default.

In [24]:
from aiida import orm
import os
os.sys.path.append("/home/jovyan/work/aiida-openbis/")
from aiida_openbis.utils import bisutils
import numpy as np
from aiida.cmdline.commands import cmd_archive
import ipywidgets as widgets

In [25]:
def find_first_workchain(node):
    """Find the first workchain in the provenance that created the structure node."""
    lastcalling = None
    if isinstance(node, orm.StructureData):
        previous_node = node.creator
    else:
        previous_node = node
    while previous_node is not None:
        lastcalling = previous_node
        previous_node = lastcalling.caller
    if lastcalling is not None:
        return lastcalling.label, lastcalling.pk, lastcalling.description
    return None, None, ""

In [56]:
# Connect to openBIS
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")

In [82]:
# New Simulation Experiment
eln_project_code = "/CARBON_NANOMATERIALS/TRIANGULENE_SPIN_CHAINS"
experiment_code = "TRIANGULENE_SPIN_CHAINS_EXP_5"
new_experiment = session.new_experiment(project = eln_project_code, code = experiment_code, type = 'DEFAULT_EXPERIMENT')
new_experiment.props["$name"] = "Simulations"
new_experiment.save()

# Create Atomistic Models collection
inventory_project_code = "/MATERIALS/ATOMISTIC_MODELS/"
collection_code = "ATOMISTIC_MODELS_EXP_1"
atomistic_models_collection = session.new_collection(project = inventory_project_code, code = collection_code, type = 'COLLECTION')
atomistic_models_collection.props["$name"] = "Atomistic Models"
atomistic_models_collection.save()

{"method": "createExperiments", "params": ["admin-231222104010009x88C4A3E43E31538AB4036CF696E144EC", [{"@type": "as.dto.experiment.create.ExperimentCreation", "code": "TRIANGULENE_SPIN_CHAINS_EXP_5", "typeId": {"@type": "as.dto.entitytype.id.EntityTypePermId", "permId": "DEFAULT_EXPERIMENT", "entityKind": "EXPERIMENT"}, "projectId": {"@type": "as.dto.project.id.ProjectIdentifier", "identifier": "/CARBON_NANOMATERIALS/TRIANGULENE_SPIN_CHAINS", "isModified": true}, "tagIds": [], "properties": {"$name": "Simulations", "$default_object_type": null, "$show_in_project_overview": null, "finished_flag": null, "default_experiment.grant": null, "start_date": null, "end_date": null, "default_experiment.experimental_goals": null, "default_experiment.experimental_description": null, "default_experiment.experimental_results": null, "reference": null, "publication": null, "notes": null, "$xmlcomments": null}}]], "id": "2", "jsonrpc": "2.0"}


ValueError: Experiment already exists in the database and needs to be unique. (Context: [])

In [84]:
all_experiments = session.get_experiments().df

selected_experiment_identifier = None

def on_dropdown_change(change):
    global selected_experiment_identifier
    selected_experiment_code = change.new
    all_experiments_codes = list(all_experiments["identifier"].values)
    experiment_index = all_experiments_codes.index(selected_experiment_code)
    selected_experiment_identifier = all_experiments["permId"].values[experiment_index]

experiment_code_dropdown = widgets.Dropdown(
    options = all_experiments["identifier"].values,
    value = all_experiments["identifier"].values[0],
    description = 'Experiment:',
    disabled = False,
)

experiment_code_dropdown.observe(on_dropdown_change, names='value')

pk_inttext = widgets.IntText(
    value = 7,
    description = 'Simulation PK:',
    disabled = False
)

send_openbis_button = widgets.Button(
    description = 'Send simulation to openBIS',
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'Click me',
    layout = widgets.Layout(width='auto')
)

def send_data_to_openbis(b):
    # Get all simulations and atomistic models from openbis
    simulations_openbis = session.get_objects(type = "SIMULATION")
    atomistic_models_openbis = session.get_objects(type = "ATOMISTIC_MODEL")

    # Get STM Simulation Workchain from AiiDA
    stm_workchain_pk = pk_inttext.value
    stm_workchain_node = load_node(stm_workchain_pk)
    structure_stm = stm_workchain_node.inputs.structure
    structure_stm_pk = structure_stm.pk
    structure_stm_node = load_node(structure_stm_pk)

    # Get Geometry Optimisation Workchain from AiiDA
    geoopt_workchain_details = find_first_workchain(structure_stm_node)
    geoopt_workchain_pk = geoopt_workchain_details[1]
    geoopt_workchain = load_node(geoopt_workchain_pk)

    # Get First Geometry from AiiDA
    first_structure = geoopt_workchain.inputs.structure
    first_structure_ase = first_structure.get_ase()
    first_structure_ase.positions # Atoms positions
    first_structure_ase.symbols # Atoms Symbols
    first_structure_ase.pbc # PBC (X, Y, Z)
    first_structure.cell # Cell vectors

    # Verify whether the non-optimised atomistic model exists
    atomistic_model_exists = False

    for atomistic_model in atomistic_models_openbis:
        if atomistic_model.props.get("wfm-uuid") == first_structure.uuid:
            first_atomistic_model = atomistic_model
            atomistic_model_exists = True
            print(f"Atomistic model {atomistic_model.props.get('wfm-uuid')} already exists.")
    
    if atomistic_model_exists == False:
        # Create Atomistic Model (Not Optimised) in openBIS
        first_atomistic_model = session.new_sample(collection = atomistic_models_collection, type='ATOMISTIC_MODEL')
        first_atomistic_model.props['$name'] = "Atomistic Model 1"
        first_atomistic_model.props['description'] = "A nice atomistic model"
        first_atomistic_model.props['wfm-uuid'] = first_structure.uuid
        first_atomistic_model.props['pbcx'] = int(first_structure_ase.pbc[0])
        first_atomistic_model.props['pbcy'] = int(first_structure_ase.pbc[1])
        first_atomistic_model.props['pbcz'] = int(first_structure_ase.pbc[2])
        first_atomistic_model.props['geo-is-optimised'] = 0
        first_atomistic_model.save()

    # Verify whether the simulation exists
    simulation_exists = False

    for simulation in simulations_openbis:
        if simulation.props.get("wfm-uuid") == geoopt_workchain.uuid:
            geoopt_simulation = simulation
            simulation_exists = True
            print(f"Simulation {simulation.props.get('wfm-uuid')} already exists.")
    
    if simulation_exists == False:
        # Create Simulation in openBIS
        geoopt_simulation = session.new_sample(experiment = selected_experiment_identifier, type='SIMULATION')
        geoopt_simulation.props['$name'] = "Simulation GeoOpt"
        geoopt_simulation.props['description'] = "Simulation Geometry Optimisation"
        # new_simulation.props['simulation-type'] = "Something"
        # new_simulation.props['wfm-connection'] = "?"
        geoopt_simulation.props['wfm-uuid'] = geoopt_workchain.uuid
        geoopt_simulation.save()

        # Create Geo Optimisation in openBIS and link it to Simulation and Atomistic Model objects
        geoopt_multiplicity = geoopt_workchain.inputs.multiplicity.value
        geo_optimisation = session.new_sample(experiment = selected_experiment_identifier, type='GEO_OPTIMISATION', parents = [geoopt_simulation, first_atomistic_model])
        geo_optimisation.props['$name'] = "Simulated Geometry Optimisation"
        geo_optimisation.props['multiplicity'] = geoopt_multiplicity
        geo_optimisation.save()

    # Get Optimised Geometry from AiiDA
    optimised_structure = stm_workchain_node.inputs.structure
    optimised_structure_ase = optimised_structure.get_ase()
    optimised_structure_ase.positions # Atoms positions
    optimised_structure_ase.symbols # Atoms Symbols
    optimised_structure_ase.pbc # PBC (X, Y, Z)
    optimised_structure.cell # Cell vectors

    # Verify whether the optimised atomistic model exists
    atomistic_model_exists = False

    for atomistic_model in atomistic_models_openbis:
        if atomistic_model.props.get("wfm-uuid") == optimised_structure.uuid:
            optimised_atomistic_model = atomistic_model
            atomistic_model_exists = True
            print(f"Atomistic model {atomistic_model.props.get('wfm-uuid')} already exists.")
    
    if atomistic_model_exists == False:
        optimised_atomistic_model = session.new_sample(collection = atomistic_models_collection, type = 'ATOMISTIC_MODEL', parents = [geo_optimisation])
        optimised_atomistic_model.props['$name'] = "Atomistic Model 2"
        optimised_atomistic_model.props['description'] = "A nice atomistic model"
        optimised_atomistic_model.props['wfm-uuid'] = optimised_structure.uuid
        optimised_atomistic_model.props['pbcx'] = int(optimised_structure_ase.pbc[0])
        optimised_atomistic_model.props['pbcy'] = int(optimised_structure_ase.pbc[1])
        optimised_atomistic_model.props['pbcz'] = int(optimised_structure_ase.pbc[2])
        optimised_atomistic_model.props['geo-is-optimised'] = 1
        optimised_atomistic_model.save()


    simulation_exists = False

    for simulation in simulations_openbis:
        if simulation.props.get("wfm-uuid") == geoopt_workchain.uuid:
            stm_simulation = simulation
            simulation_exists = True
            print(f"Simulation {simulation.props.get('wfm-uuid')} already exists.")
    
    if simulation_exists == False:
        # Simulated Scanning Tunneling Microscopy
        stm_simulation = session.new_sample(experiment = selected_experiment_identifier, type = 'SIMULATION')
        stm_simulation.props['$name'] = "Simulation STM"
        stm_simulation.props['description'] = "Simulation"
        stm_simulation.props['wfm-uuid'] = stm_workchain_node.uuid
        stm_simulation.save()

        stm_simulation_openbis = session.new_sample(experiment = selected_experiment_identifier, type = 'STM_SIMULATION', parents = [stm_simulation, optimised_atomistic_model])
        simulated_stm_uuid = stm_workchain_node.uuid
        dft_params = dict(stm_workchain_node.inputs.dft_params)
        stm_params = dict(stm_workchain_node.inputs.stm_params)
        stm_simulation_openbis.props['$name'] = "Simulated STM"
        stm_simulation_openbis.props['description'] = "A nice simulated STM"
        stm_simulation_openbis.props['e-min'] = stm_params['--energy_range'][0]
        stm_simulation_openbis.props['e-max'] = stm_params['--energy_range'][1]
        stm_simulation_openbis.props['de'] = stm_params['--energy_range'][2]
        stm_simulation_openbis.save()

        #TODO: How to do this using Python commands?
        stm_simulation_dataset_filename = "stm_simulation.aiida"
        os.system(f"verdi archive create {stm_simulation_dataset_filename} --no-call-calc-backward --no-call-work-backward --no-create-backward -N {stm_workchain_pk}")

        stm_simulation_dataset = session.new_dataset(
            type = "RAW_DATA", 
            files = ['/home/jovyan/work/aiida-openbis/Python_Scripts/stm_simulation.aiida'],
            sample = stm_simulation_openbis
        )
        stm_simulation_dataset.save()

send_openbis_button.on_click(send_data_to_openbis)
display(experiment_code_dropdown)
display(pk_inttext)
display(send_openbis_button)

Dropdown(description='Experiment:', options=('/DEFAULT/DEFAULT/DEFAULT', '/ELN_SETTINGS/STORAGES/STORAGES_COLL…

IntText(value=7, description='Simulation PK:')

Button(description='Send simulation to openBIS', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Cl…

Atomistic model 11cbe820-f585-422d-ad75-c09849b7ed68 already exists.
sample successfully created.
sample successfully created.
sample successfully created.
sample successfully created.
sample successfully created.


Critical: failed to write the archive file. Exception: The output file 'stm_simulation.aiida' already exists


https://openbis/
DataSet successfully created.
Atomistic model 11cbe820-f585-422d-ad75-c09849b7ed68 already exists.
Simulation c2a29793-2201-4704-ae80-f8a2b4be63b3 already exists.
Atomistic model 5bbc1f32-3000-4c57-83e1-135cc4701410 already exists.
Simulation c2a29793-2201-4704-ae80-f8a2b4be63b3 already exists.


In [6]:
# Code that may be useful in future

# pk = 62
# workchain = load_node(pk)
# structure = workchain.inputs.structure

# all_structures = [structure]
# creator_found = True

# while creator_found:
#     creator = structure.creator
#     if creator is not None:
#         structure = creator.inputs.structure
#         all_structures.append(structure)
#     else:
#         creator_found = False